In [1]:
%run nb-env.ipynb
pd.set_option("display.precision", 2)

# TODO: check dates

[2022-05-05T09:00:58.589050+08:00] Enviroment loaded. Working Dir: /Users/tzhu/work/lab/neo_world/notebooks
Today is 2022-05-05, Working from 2021-04-13 to 2022-04-29


In [2]:
%%time

today_rs = pdl.now().to_date_string()
# today_rs = '2022-04-29'
# end_date = '2022-04-28'

dc, df_init = init_data(biquater_ago_date, end_date, expire_days=5)
df = df_init.copy()
clean_cache_files()

Initializing data from 2021-09-29 to 2022-04-29...
Found cache file: ../tmp/price_2021-09-29_2022-04-29_2022-05-04.feather, loading...
Preserving recent file price_2018-02-01_2021-04-22_2023-02-26.feather
Preserving recent file price_2018-08-02_2021-04-22_2023-02-26.feather
Preserving recent file price_2020-01-29_2021-10-29_2022-12-31.feather
Preserving recent file price_2021-09-29_2022-04-28_2022-05-09.feather
Deleting old file price_2021-09-29_2022-04-29_2022-05-04.feather
CPU times: user 1.96 s, sys: 1.43 s, total: 3.39 s
Wall time: 3.62 s


In [3]:
# 昨日指标
cols = ['name', 'close', 'ma_close_60', 'conseq_up_num', 'circ_mv', 'total_mv', 
        'turnover_rate_f', 'vol_ratio', 'amount', 'open_pct', 'pct_chg', 'up_type',
        'first_time', 'last_time', 'strth', 'next_auc_amt', 'next_open_pct', 
        'next_pct_chg', 'next2_pct_chg']

s = Strategy(name='xfe')
s.add_condition('pct_chg', '>', val=9)
s.add_condition('conseq_up_num', '>=', val=1)
s.add_condition('conseq_up_num', '<=', val=3)
s.add_condition('conseq_up_num', '<', var='list_days')
s.add_condition('amount',  '<', val=1500000) # 15亿，unit: k
# s.add_condition('circ_mv', '<', val=1000000) # 80亿，unit: w
s.add_condition('turnover_rate_f', '<', val=45)
s.add_condition('turnover_rate_f', '>=', val=5)
s.add_condition('high', '>', var='low', ratio=1.02) # 日内振幅大于 2% (非一字板）
# s.add_condition('vol_ratio', '>', val=1.5)
# s.add_condition('vol_ratio', '<', val=3.3)
# s.add_condition('vol', '>', var='ma_vol_5', ratio=1.7)

# 近期涨幅
s.add_condition('close', '<', var='ma_close_60', ratio=1.2)

In [4]:
s.stock_filter = StockFilter(end_date).not_st().zb()
df2, a = s.get_result(df=df, trade_date=end_date)
print(f'Get {len(df2)} records from 1st filter...')

Searching 2022-04-29...
Get 73 records from 1st filter...


In [5]:
# 获取竞价数据（Run this after trading day 9:25）
auc = ak_today_auctions(ts_codes=df2.index)
auc1 = auc.rename(columns={'auc_amt':'next_auc_amt', 'open':'next_open'}).droplevel('trade_date')[['next_auc_amt','next_open']]

# 合并竞价数据
df3 = df2.drop(columns=['next_auc_amt']).join(auc1)
df3.next_open_pct = round((df3.next_open/df3.close-1)*100,2)

  0%|          | 0/73 [00:00<?, ?it/s]

In [9]:
display_up_df(df3[cols].sort_values('next_auc_amt', ascending=False))

,name,close,ma_close_60,连板数,流值,总市值,流动换手率,vol_ratio,成交额,开盘涨幅,涨幅,up_type,first_time,last_time,strth,次日竞价金额,次开涨幅,次日涨幅,next2_pct_chg
ts_code,,,,,,,,,,,,,,,,,,,
600559.SH,老白干酒,21.51,21.96,1,193.01亿,193.01亿,9.35%,2.78,13.34亿,7.47%,10.03%,O,09:34:48,11:05:30,55.93,0.38亿,2.28%,nan%,nan
601933.SH,永辉超市,4.43,4.06,1,402.02亿,402.02亿,5.91%,1.34,8.87亿,0.00%,9.93%,W,14:37:19,14:51:34,49.60,0.37亿,2.71%,nan%,nan
002349.SZ,精华制药,13.89,16.07,1,112.69亿,113.09亿,19.60%,1.01,9.49亿,0.40%,9.98%,O,10:00:21,13:14:33,51.27,0.36亿,2.23%,nan%,nan
000665.SZ,湖北广电,7.55,7.87,1,85.46亿,85.46亿,19.59%,0.92,8.84亿,2.04%,10.06%,O,10:10:27,10:10:27,56.50,0.30亿,1.32%,nan%,nan
000656.SZ,金科股份,4.46,4.61,1,236.91亿,238.15亿,8.16%,1.21,11.82亿,0.00%,10.12%,W,14:23:30,14:34:54,48.04,0.29亿,-0.90%,nan%,nan
600243.SH,青海华鼎,5.04,4.30,2,22.12亿,22.12亿,15.58%,1.02,2.61亿,6.77%,10.04%,O,09:32:31,09:32:31,65.51,0.26亿,5.16%,nan%,nan
002061.SZ,浙江交科,6.80,6.51,1,104.48亿,104.48亿,16.24%,2.30,8.11亿,6.47%,10.03%,O,10:09:06,14:01:06,52.68,0.25亿,2.65%,nan%,nan
603329.SH,上海雅仕,16.19,14.27,3,25.03亿,25.70亿,41.66%,2.49,5.75亿,1.90%,9.99%,O,09:59:07,14:50:15,55.74,0.24亿,-0.31%,nan%,nan
600838.SH,上海九百,8.06,6.89,1,32.31亿,32.31亿,7.23%,0.70,1.65亿,-1.77%,9.96%,L,09:38:11,09:38:11,54.04,0.23亿,2.61%,nan%,nan


In [6]:
# 添加竞价过滤条件
s.add_condition('next_open_pct', '<', val=7)
s.add_condition('next_open_pct', '>', val=2)
s.add_condition('next_auc_amt', '>', var='amount', ratio=0.05)
# s.add_condition('next_auc_amt', '<', var='pre_amount', ratio=0.05)
s.add_condition('next_auc_amt', '>', val=10000)
s.add_condition('next_auc_amt', '<', val=100000)

In [7]:
# 得到最终数据
df4, a = s.get_result(df=df3)
display_up_df(df4[cols])

,name,close,ma_close_60,连板数,流值,总市值,流动换手率,vol_ratio,成交额,开盘涨幅,涨幅,up_type,first_time,last_time,strth,次日竞价金额,次开涨幅,次日涨幅,next2_pct_chg
ts_code,,,,,,,,,,,,,,,,,,,
000548.SZ,湖南投资,4.66,4.35,1,23.26亿,23.26亿,11.45%,2.41,1.70亿,0.00%,9.91%,W,13:26:03,13:26:03,55.79,0.11亿,2.15%,nan%,nan
600162.SH,香江控股,2.26,2.27,1,76.74亿,76.74亿,6.22%,1.81,1.65亿,1.46%,10.24%,W,10:34:57,10:34:57,55.53,0.13亿,4.87%,nan%,nan
600243.SH,青海华鼎,5.04,4.30,2,22.12亿,22.12亿,15.58%,1.02,2.61亿,6.77%,10.04%,O,09:32:31,09:32:31,65.51,0.26亿,5.16%,nan%,nan
600838.SH,上海九百,8.06,6.89,1,32.31亿,32.31亿,7.23%,0.70,1.65亿,-1.77%,9.96%,L,09:38:11,09:38:11,54.04,0.23亿,2.61%,nan%,nan
